检查环境

In [1]:
import os
import torch
import cv2 as cv

!whoami
!ls
!pwd
!touch test
# os.chdir('drive')
# os.chdir('My Drive')
!pwd
print(torch.cuda.is_available())
print(torch.cuda.device_count())
print(torch.cuda.get_device_name(torch.device('cuda:0')))
print(torch.device('cuda:0'))
print(torch.cuda.get_device_name(0))
print(torch.cuda.current_device())



root
drive  sample_data
/content
/content
True
1
Tesla K80
cuda:0
Tesla K80
0


拷贝数据集到本地

In [5]:
import os
path_origin = os.path.join(os.getcwd(),'drive','My Drive', 'Colab', 'extract', '1920x980')
print(os.path.exists(path_origin))
files = os.listdir(os.path.join(path_origin))
if not os.path.exists('dataset'):
    os.mkdir('dataset')
path_target = os.path.join(os.getcwd(), 'dataset')
index = 0
for image in files:
    if image is "fighting (259).png":
        continue
    index = index + 1
    with open(os.path.join(path_origin, image), 'rb') as f:
        print(f"read index {index}/{len(files)}")
        data = f.read()
        with open(os.path.join(path_target, image), 'wb') as w:
            print(f"write index {index}/{len(files)}")
            w.write(data)
print("finished")

True
read index 1/3292
write index 1/3292
read index 2/3292
write index 2/3292
read index 3/3292
write index 3/3292
read index 4/3292
write index 4/3292
read index 5/3292
write index 5/3292
read index 6/3292
write index 6/3292
read index 7/3292
write index 7/3292
read index 8/3292
write index 8/3292
read index 9/3292
write index 9/3292
read index 10/3292
write index 10/3292
read index 11/3292
write index 11/3292
read index 12/3292
write index 12/3292
read index 13/3292
write index 13/3292
read index 14/3292
write index 14/3292
read index 15/3292
write index 15/3292
read index 16/3292
write index 16/3292
read index 17/3292
write index 17/3292
read index 18/3292
write index 18/3292
read index 19/3292
write index 19/3292
read index 20/3292
write index 20/3292
read index 21/3292
write index 21/3292
read index 22/3292
write index 22/3292
read index 23/3292
write index 23/3292
read index 24/3292
write index 24/3292
read index 25/3292
write index 25/3292
read index 26/3292
write index 26/3292

KeyboardInterrupt: ignored

清理已下载的数据

In [7]:
files = os.listdir('dataset')
for file in files:
    os.remove(os.path.join(os.getcwd(), 'dataset', file))
os.rmdir('dataset')

可选：从数据集解压数据到本地

In [9]:
import zipfile
import os

path_origin = os.path.join(os.getcwd(), 'drive', 'My Drive', 'Colab', 'dataset.zip')
path_target = os.path.join(os.getcwd(), 'extract')
if not os.path.exists(path_target):
    os.mkdir(path_target)

fz = zipfile.ZipFile(path_origin, 'r')
for file in fz.namelist():
    fz.extract(file, path_target)

定义模型和学习率

In [11]:
learn_rate = 0.1

class TestModule(torch.nn.Module):
    def __init__(self):
        super().__init__()
        # 使用三层
        #   - 卷积层（同时实现不同尺寸图像的输入）
        #   - 池化层（缩小图像尺寸）
        #   - 全连接层
        #   - 分类器输出
        self.conv1 = torch.nn.Conv2d(1, 16, kernel_size=3, stride=2, padding=1)
        self.conv2 = torch.nn.Conv2d(16, 16, kernel_size=3, stride=2, padding=1)
        self.conv3 = torch.nn.Conv2d(16, 10, kernel_size=3, stride=2, padding=1)
        self.adaptive_avg_pool = torch.nn.AdaptiveAvgPool2d([16, 36])
        self.fc = torch.nn.Linear(10*16*36, 8)

    def forward(self, xb):

        # xb = torch.nn.functional.adaptive_avg_pool2d(xb, 4)
        # xb = xb.view(-1, 1, 980, 1920)
        # xb = xb.unsqueeze(0)
        # xb = xb.unsqueeze(0)
        # xb = self.conv1(xb)
        # xb = self.conv2(xb)
        # xb = self.conv3(xb)
        xb = torch.nn.functional.relu(self.conv1(xb))
        xb = torch.nn.functional.relu(self.conv2(xb))
        xb = torch.nn.functional.relu(self.conv3(xb))
        xb = self.adaptive_avg_pool(xb)
        xb = xb.view(-1, 10*16*36)
        xb = torch.sigmoid(self.fc(xb))

        # return xb.view(-1, xb.size(1))
        return xb


test_module = TestModule()
test_module.to(torch.device('cuda'))
optimize = torch.optim.SGD(test_module.parameters(), lr=learn_rate)


配置 DataSet

In [12]:
# dataset_path = os.path.join(os.getcwd(), 'drive', 'My Drive', 'Colab',"extract", "1920x980")
# dataset_path = os.path.join(os.getcwd(), 'dataset')
dataset_path = os.path.join(os.getcwd(), 'extract', '1920x980')
file_list = os.listdir(dataset_path)
tag_dic = ["level_selection",
           "restore_sanity_medicine",
           "restore_sanity_stone",
           "team_up",
           "fighting",
           "battle_settlement",
           "annihilation_settlement",
           "level_up"]

def map_func(filename:str) -> torch.Tensor:
    for index, tag in enumerate(tag_dic):
        if tag in filename:
            return torch.tensor(index, dtype=torch.long, device=torch.device('cuda'))

tag_list = list(map(map_func, file_list))

print(len(file_list))
print(file_list[0])
print(tag_dic)
print(len(tag_list))
print(tag_list[2])

from torch.utils.data import Dataset
# from torch.utils.data.dataset import T_co


def default_loader(image_path:str):
    image_loaded = cv.imread(
        os.path.join(dataset_path, image_path),
        cv.IMREAD_GRAYSCALE)
    image_tensor = torch.tensor(image_loaded, device=torch.device('cuda'), dtype=torch.float32).unsqueeze(0)
    print(f"load image {image_path}")
    # print(image_tensor.size())
    return image_tensor

# imgae_list = list(map(default_loader, file_list))

# def custom_loader(image_path:str):
#     return image_list[index].to(torch.d)

class TrainSet(Dataset):
    def __init__(self, image_name_list:list, targets:int, loader=default_loader)-> None:
        self.images = image_name_list
        self.targets = targets
        self.loader = loader
    def __getitem__(self, index: int):
        data = self.loader(self.images[index])
        target = self.targets[index]
        return data, target
    def __len__(self)-> int:
        return len(self.images)

train_set = TrainSet(file_list, tag_list)
print(train_set)
print(train_set.__getitem__(3))
print(train_set.__len__())

3292
fighting (927).png
['level_selection', 'restore_sanity_medicine', 'restore_sanity_stone', 'team_up', 'fighting', 'battle_settlement', 'annihilation_settlement', 'level_up']
3292
tensor(4, device='cuda:0')
load image level_selection (34).png
(tensor([[[115., 116., 116.,  ...,  35.,  35.,  35.],
         [113., 114., 115.,  ...,  36.,  36.,  36.],
         [112., 114., 115.,  ...,  36.,  36.,  36.],
         ...,
         [ 71.,  72.,  73.,  ...,  36.,  36.,  36.],
         [ 71.,  72.,  73.,  ...,  36.,  36.,  36.],
         [ 71.,  72.,  73.,  ...,  36.,  36.,  36.]]], device='cuda:0'), tensor(0, device='cuda:0'))
3292


构建 DataLoader

In [13]:
from torch.utils.data import DataLoader
train_loader = DataLoader(train_set, batch_size=1, shuffle=True)
print(train_loader)
print(iter(train_loader).next()[0].size())
print(iter(train_loader).next()[1].size())

load image team_up (257).png
torch.Size([1, 1, 980, 1920])
load image level_selection (245).png
torch.Size([1])


Cost 测试

In [14]:
loss_function = torch.nn.functional.cross_entropy
with torch.no_grad():
    pred_test = test_module(iter(train_loader).next()[0])
    yb = iter(train_loader).next()[1]
    print(pred_test.size())
    print(yb.size())
    print(loss_function(pred_test,yb))

load image team_up (264).png
load image team_up (402).png
torch.Size([1, 8])
torch.Size([1])
tensor(1.9935, device='cuda:0')


准备 fit 函数

In [15]:
def fit(epochs:int):
    index = 0
    for epoch in range(epochs):
        for xb, yb in train_loader:
            index = index + 1
            if index%50 is 0:
                print(f"start index {index} data")
            pred = test_module(xb)
            loss = loss_function(pred, yb)

            loss.backward()
            optimize.step()
            optimize.zero_grad()

    xb, yb = train_set.__getitem__(30)
    print(loss_function(test_module(xb), yb))

fit(1)

load image level_selection (549).png
load image team_up (94).png
load image level_selection (248).png
load image battle_settlement (608).png
load image team_up (278).png
load image fighting (90).png
load image team_up (493).png
load image level_selection (828).png
load image battle_settlement (168).png
load image level_selection (861).png
load image level_selection (795).png
load image battle_settlement (539).png
load image team_up (344).png
load image fighting (433).png
load image level_selection (486).png
load image fighting (1000).png
load image level_selection (332).png
load image battle_settlement (295).png
load image level_selection (416).png
load image team_up (34).png
load image team_up (384).png
load image fighting (471).png
load image team_up (138).png
load image level_selection (275).png
load image level_selection (472).png
load image battle_settlement (224).png
load image fighting (196).png
load image restore_sanity_medicine (88).png
load image level_selection (279).png
loa

RuntimeError: ignored